# 위도 경도 변환
경도,위도의 QGIS에서의 버퍼처리를 편하게 하기 위해 Angular Coordinate 에서 Projectile Coordinate로 변경합니다. 

In [4]:
import pandas as pd
from glob import glob

In [5]:
PATH_STATION_ABS_POS=glob("../raw_data/selected/국가철도공단_부산*호선_역위치*.csv")
PATH_STATION_ABS_POS_REFINE = "../trimmed_data/stationAbsoluteLocation.csv"

In [6]:
# 경도위도 정보 불러오기
df_abs_pos = [pd.read_csv(path,encoding="EUC-KR") for path in PATH_STATION_ABS_POS]
df_abs_pos = pd.concat(df_abs_pos)
df_abs_pos = df_abs_pos.reset_index()
df_abs_pos = df_abs_pos.loc[:,"선명":"위도"]
df_abs_pos.head()

,선명,역명,경도,위도
0,1호선,동매,128.974200,35.089900
1,1호선,신평,128.960564,35.095179
2,1호선,하단(부산본병원),128.966803,35.106180
3,1호선,당리(사하구청),128.973846,35.103532
4,1호선,사하,128.983100,35.099847


In [7]:
# 여태 사용한 데이터와 같은 형식을 갖도록 역명을 편집
df_abs_pos.loc[:,"역명"] = df_abs_pos.loc[:,"역명"].apply(lambda _X:_X.split("(")[0])
df_abs_pos.loc[:,"역명"] = df_abs_pos.loc[:,"역명"].apply(lambda _X:_X.replace("·","."))
df_abs_pos.loc[:,"역명"] = df_abs_pos.loc[:,"역명"].apply(lambda _X:_X.replace("국제금융센터","국제금융"))
df_abs_pos.head()

,선명,역명,경도,위도
0,1호선,동매,128.974200,35.089900
1,1호선,신평,128.960564,35.095179
2,1호선,하단,128.966803,35.106180
3,1호선,당리,128.973846,35.103532
4,1호선,사하,128.983100,35.099847


In [8]:
# GIS 처리를 수월하게 하기 위해 일반 위도경도좌표계(EPSG:4326) 에서 한국향사영좌표계(ESPG:5179)로 좌표계 변환 진행
# 아래의 레퍼런스 기반으로 작성됨.
# https://wooiljeong.github.io/python/transform_coord/
# https://thlee33.medium.com/%EC%A2%8C%ED%91%9C%EB%B3%80%ED%99%98%EC%97%90-%EB%AC%B4%EC%8A%A8-%ED%8C%A8%ED%82%A4%EC%A7%80%EB%A5%BC-%EC%93%B0%EB%A9%B4-%EC%A2%8B%EC%9D%84%EA%B9%8C-2023-06-556be8fb82d8
import pyproj

# 프로젝션 정의
wgs84 = pyproj.CRS("epsg:4326")  # 원래 좌표계 (WGS84)
tm_korea = pyproj.CRS("epsg:5179")  # 변환하려는 좌표계 (TM Korea)

# Transformer 생성
transformer = pyproj.Transformer.from_crs(wgs84, tm_korea)

# lon:경도,x lat:위도,y
# 좌표 변환 함수 정의
def convert_coordinates(lon, lat ):
    new_lat, new_lon = transformer.transform(
        lat, lon
    )  # 주의: transform 함수는 위도 먼저 받습니다.
    return new_lon, new_lat

# 새로운 좌표를 계산하고 새로운 컬럼에 저장
df_abs_pos_transformed = pd.DataFrame(convert_coordinates(df_abs_pos.loc[:,"경도"],df_abs_pos.loc[:,"위도"])).T
df_abs_pos_transformed = df_abs_pos_transformed.rename(columns={0:"경도",1:"위도"})

# 변황된 좌표계 입력하기
df_abs_pos.loc[:,"경도":"위도"] = df_abs_pos_transformed.loc[:,"경도":"위도"]
df_abs_pos.head()

,선명,역명,경도,위도
0,1호선,동매,1.134381e+06,1.678192e+06
1,1호선,신평,1.133129e+06,1.678759e+06
2,1호선,하단,1.133680e+06,1.679987e+06
3,1호선,당리,1.134326e+06,1.679703e+06
4,1호선,사하,1.135176e+06,1.679307e+06


In [9]:
with open(PATH_STATION_ABS_POS_REFINE,mode="w",encoding="UTF-8") as f:
    f.write(df_abs_pos.to_csv(index=False))